# 단어 Level로 번역기 만들기

지난 실습때, ENG2FRA_TRANSLATOR를 문자 수준의 번역기로 구현해보는 작업을 해보았습니다.  

이번에는 단어 수준의 번역기를 만들어보는 작업을 해보도록 하겠습니다.  

프랑스어와 영어의 병렬코퍼스는 다음의 사이트에서 받았습니다.  

https://www.manythings.org/anki/fra-eng.zip  

## Step 1. 데이터 전처리하기(영어, 프랑스어 모두!)

In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

In [3]:
import os
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
dictionary = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
# \t에 따라 라벨링하여 분류하기
print('전체 샘플의 수 :',len(dictionary))
dictionary.sample(5) #샘플 5개 출력

전체 샘플의 수 : 178009


,eng,fra,cc
53947,Don't tell Tom anything.,Ne dites rien à Tom.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
137092,Tom and Mary returned to their seats.,Tom et Mary retournèrent à leurs places.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
116974,I want to know why this happened.,Je veux savoir pourquoi ceci a eu lieu.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
76480,I did none of those things.,Je n'ai fait aucune de ces choses.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
156314,I want you to know that I'll work very hard.,Je veux que vous sachiez que je travaillerez v...,CC-BY 2.0 (France) Attribution: tatoeba.org #7...



이번 실습에서는 33000개의 샘플중 30000개를 train set으로, 3000개를 test set으로 사용하도록 하겠습니다.

In [4]:
dictionary = dictionary[['eng', 'fra']][:33000] # 33000개 샘플 사용
dictionary.sample(5)

,eng,fra
24816,I was busy cooking.,J'étais occupée à faire la cuisine.
4087,Is Tom right?,Est-ce que Tom a raison ?
17690,You must tell me.,Il vous faut me le dire.
11355,"I think so, too.",Je le pense aussi.
23738,He sat next to her.,Il s'est assis à son côté.


다음과 같은 지시사항을 지켜주기 위해 함수를 만들어 작업을 하였습니다.  

1. 구두점(Punctuation)을 단어와 분리해주세요.
2. 소문자로 바꿔주세요.  
3. 띄어쓰기 단위로 토큰를 수행하세요.  

In [5]:
import re
def encoder_preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)    # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)           # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # 해당 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    sentence = sentence.strip()
    sentence = sentence.split(" ")
    
    return sentence

디코더의 경우, 시작점과 끝점을 알려주는 것이 중요함으로 문장 앞 뒤에 단어들을 추가하였습니다.

In [6]:
def decoder_preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)    # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)           # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # 해당 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'            #문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    sentence = sentence.split(" ")
    
    return sentence

In [7]:
dictionary.eng = dictionary.eng.apply(lambda x : encoder_preprocess_sentence(x))

In [8]:
dictionary.fra = dictionary.fra.apply(lambda x : decoder_preprocess_sentence(x))

In [9]:
dictionary.eng.sample(5)

10072          [are, you, excited, ?]
23282     [don, t, do, that, here, .]
32155    [this, guy, is, a, loser, .]
2485               [i, m, engaged, .]
27884     [you, were, bad, at, it, .]
Name: eng, dtype: object

In [10]:
dictionary.eng

0                            [go, .]
1                            [hi, .]
2                            [hi, .]
3                           [run, !]
4                           [run, !]
                    ...             
32995    [what, was, their, goal, ?]
32996    [what, were, you, doing, ?]
32997    [what, would, tom, need, ?]
32998    [what, would, you, like, ?]
32999    [what, would, you, like, ?]
Name: eng, Length: 33000, dtype: object

In [11]:
dictionary.fra.sample(5)

29947    [<start>, j, adore, cette, entreprise, ., <end>]
23653      [<start>, c, est, un, homme, simple, ., <end>]
18168              [<start>, sais, tu, lire, a, ?, <end>]
4538                 [<start>, tom, adorait, a, ., <end>]
20486           [<start>, faisons, un, g, teau, ., <end>]
Name: fra, dtype: object

In [12]:
dictionary.fra

0                                  [<start>, va, !, <end>]
1                               [<start>, salut, !, <end>]
2                               [<start>, salut, ., <end>]
3                               [<start>, cours, !, <end>]
4                              [<start>, courez, !, <end>]
                               ...                        
32995           [<start>, quel, tait, leur, but, ?, <end>]
32996    [<start>, qu, tais, tu, en, train, de, faire, ...
32997    [<start>, de, quoi, tom, aurait, il, besoin, ?...
32998                [<start>, qu, aimerais, tu, ?, <end>]
32999              [<start>, qu, aimeriez, vous, ?, <end>]
Name: fra, Length: 33000, dtype: object

### Text 토큰화하기

In [13]:
eng_tokenizer = Tokenizer()                         # 단어 단위로 Tokenizer를 생성합니다. 
eng_tokenizer.fit_on_texts(dictionary.eng)               # 33000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(dictionary.eng)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

[[30, 1], [1133, 1], [1133, 1]]

In [14]:
fra_tokenizer = Tokenizer()                         # 단어 단위로 Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(dictionary.fra)               # 33000개의 행을 가진 eng의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(dictionary.fra)    # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[1, 92, 12, 2], [1, 1069, 12, 2], [1, 1069, 3, 2]]

In [15]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1

max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])

In [16]:
print('전체 샘플의 수 :',len(dictionary))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 4663
프랑스어 단어장의 크기 : 7327
영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 17


In [17]:
sos_token = '<start>'
eos_token = '<end>'

디코더의 인풋 데이터와 타겟 데이터에서 학습을 위해 각각 종료 토큰과 시작 토큰을 제거해주었습니다.

In [18]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [19]:
print(decoder_input[:3])
print(decoder_target[:3])

[[1, 92, 12], [1, 1069, 12], [1, 1069, 3]]
[[92, 12, 2], [1069, 12, 2], [1069, 3, 2]]


In [20]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 8)
프랑스어 입력데이터의 크기(shape) : (33000, 17)
프랑스어 출력데이터의 크기(shape) : (33000, 17)


In [21]:
print(encoder_input[0])

[30  1  0  0  0  0  0  0]


In [22]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input_train))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input_train))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target_train))

영어 학습데이터의 크기(shape) : (30000, 8)
프랑스어 학습 입력데이터의 크기(shape) : (30000, 17)
프랑스어 학습 출력데이터의 크기(shape) : (30000, 17)


### 케라스 임베딩 레이어를 사용하여 모델 구현하기

In [23]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [24]:
embedding_size = 128
hidden_size = 128

encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(eng_vocab_size, embedding_size)(encoder_inputs)
enc_masking = Masking(mask_value=0.0)(enc_emb)
encoder_lstm = LSTM(hidden_size, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

In [25]:
# 입력 텐서 생성.
decoder_inputs = Input(shape=(None,))
dec_emb =  Embedding(fra_vocab_size, embedding_size)(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)
decoder_lstm = LSTM(hidden_size, return_sequences = True, return_state=True)
# decoder_outputs는 모든 time step의 hidden state
decoder_outputs, _, _= decoder_lstm(dec_masking, initial_state = encoder_states)

In [26]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [27]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [28]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 128)    596864      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 128)    937856      input_2[0][0]                    
______________________________________________________________________________________________

In [29]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
BATCH_SIZE = 64
EPOCHS = 100
filename = 'checkpoint-epoch-{}-batch-{}-trial-001.h5'.format(EPOCHS, BATCH_SIZE)
checkpoint = ModelCheckpoint(filename,             # file명을 지정합니다
                             monitor='val_loss',   # val_loss 값이 개선되었을때 호출됩니다
                             verbose=1,            # 로그를 출력합니다
                             save_best_only=True,  # 가장 best 값만 저장합니다
                             mode='auto'           # auto는 알아서 best를 찾습니다. min/max
                            )
earlystopping = EarlyStopping(monitor='val_loss',  # 모니터 기준 설정 (val loss) 
                              patience=20,         # 20회 Epoch동안 개선되지 않는다면 종료
                             )

In [30]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
# 콜백 정의
reduceLR = ReduceLROnPlateau(
    monitor='val_loss',  # 검증 손실을 기준으로 callback이 호출됩니다
    factor=0.5,          # callback 호출시 학습률을 1/2로 줄입니다
    patience=10,         # epoch 10 동안 개선되지 않으면 callback이 호출됩니다
)

In [31]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[checkpoint, earlystopping, reduceLR])

Epoch 1/100
466/469 [============================>.] - ETA: 0s - loss: 1.8705
Epoch 00001: val_loss improved from inf to 1.78417, saving model to checkpoint-epoch-100-batch-64-trial-001.h5
469/469 [==============================] - 10s 21ms/step - loss: 1.8682 - val_loss: 1.7842 - lr: 0.0010
Epoch 2/100
468/469 [============================>.] - ETA: 0s - loss: 1.3510
Epoch 00002: val_loss improved from 1.78417 to 1.57393, saving model to checkpoint-epoch-100-batch-64-trial-001.h5
469/469 [==============================] - 8s 17ms/step - loss: 1.3510 - val_loss: 1.5739 - lr: 0.0010
Epoch 3/100
468/469 [============================>.] - ETA: 0s - loss: 1.1905
Epoch 00003: val_loss improved from 1.57393 to 1.46059, saving model to checkpoint-epoch-100-batch-64-trial-001.h5
469/469 [==============================] - 8s 17ms/step - loss: 1.1903 - val_loss: 1.4606 - lr: 0.0010
Epoch 4/100
467/469 [============================>.] - ETA: 0s - loss: 1.0910
Epoch 00004: val_loss improved from 1

Epoch 32/100
469/469 [==============================] - ETA: 0s - loss: 0.4925
Epoch 00032: val_loss did not improve from 1.11398
469/469 [==============================] - 8s 18ms/step - loss: 0.4925 - val_loss: 1.1431 - lr: 5.0000e-04
Epoch 33/100
469/469 [==============================] - ETA: 0s - loss: 0.4859
Epoch 00033: val_loss did not improve from 1.11398
469/469 [==============================] - 8s 18ms/step - loss: 0.4859 - val_loss: 1.1457 - lr: 5.0000e-04
Epoch 34/100
468/469 [============================>.] - ETA: 0s - loss: 0.4798
Epoch 00034: val_loss did not improve from 1.11398
469/469 [==============================] - 8s 17ms/step - loss: 0.4799 - val_loss: 1.1451 - lr: 5.0000e-04
Epoch 35/100
468/469 [============================>.] - ETA: 0s - loss: 0.4756
Epoch 00035: val_loss did not improve from 1.11398
469/469 [==============================] - 8s 17ms/step - loss: 0.4755 - val_loss: 1.1460 - lr: 5.0000e-04
Epoch 36/100
469/469 [==============================

# 모델 평가하기

테스트 단계에서 모델이 한번에 예측 문장을 발표하는것이 아니기때문에 모델을 재조정해주어야 합니다.

### 인코더 설계하기

In [32]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 128)         596864    
_________________________________________________________________
masking (Masking)            (None, None, 128)         0         
_________________________________________________________________
lstm (LSTM)                  [(None, 128), (None, 128) 131584    
Total params: 728,448
Trainable params: 728,448
Non-trainable params: 0
_________________________________________________________________


### 디코더 설계하기

In [33]:
# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(hidden_size,))
# 이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape=(hidden_size,))
# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# decoder_states_inputs를 현재 time step의 초기 상태로 사용.
# 구체적인 동작 자체는 def decode_sequence()에 구현.
decoder_outputs, state_h, state_c = decoder_lstm(dec_emb, initial_state = decoder_states_inputs)
# 현재 time step의 hidden state와 cell state를 하나의 변수에 저장.
decoder_states = [state_h, state_c]

### 디코더 출력층 재설계하기

In [34]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 128)    937856      input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 128)]        0                                            
____________________________________________________________________________________________

### 사전 준비하기

In [35]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [36]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = fra2idx['<start>']
    
    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '<end>' or
           len(encoder_preprocess_sentence(decoded_sentence)) > max_fra_seq_len):
            stop_condition = True
            break
            
        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += ' ' + sampled_char
            

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

### 출력결과 테스트하기

In [41]:
import numpy as np
for seq_index in [100,200,3000,4321,5050]: # 입력 문장의 인덱스 (자유롭게 선택해 보세요)
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', dictionary.eng[seq_index])
    print('정답 문장:', dictionary.fra[seq_index]) 
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)]) 

-----------------------------------
입력 문장: ['call', 'us', '.']
정답 문장: ['<start>', 'appelez', 'nous', '!', '<end>']
번역기가 번역한 문장:  appelle nous !
-----------------------------------
입력 문장: ['show', 'me', '.']
정답 문장: ['<start>', 'montre', 'moi', '!', '<end>']
번역기가 번역한 문장:  montre moi !
-----------------------------------
입력 문장: ['we', 'know', 'him', '.']
정답 문장: ['<start>', 'on', 'le', 'conna', 't', '.', '<end>']
번역기가 번역한 문장:  nous le savons .
-----------------------------------
입력 문장: ['she', 'helps', 'us', '.']
정답 문장: ['<start>', 'elle', 'nous', 'aide', '.', '<end>']
번역기가 번역한 문장:  elle nous aide .
-----------------------------------
입력 문장: ['do', 'what', 'i', 'say', '.']
정답 문장: ['<start>', 'fais', 'ce', 'que', 'je', 'dis', '.', '<end>']
번역기가 번역한 문장:  fais ce que je dis .


깔끔하게 번역이 된 것 같습니다..!!

# 결과 및 분석

프랑스어 와 영어의 병렬 코퍼스를 사용하여 ENG2FRA 번역기를 만들어 보았습니다.  
모델을 학습시키는 과정에서 메모리의 부족현상이 일어나는 바람에 여러가지 loss의 종류에 대해 다시 복습하는 좋은 기회가 되었던것 같습니다.  
마지막 부분에서 번역기가 번역한 'nous le savons'라는 문장은 구글번역기를 통해 다시 영어로 번역해 보았더니 we know it이라는 뜻이 었습니다. 또 여러가지 문장들의 해석 결과를 보니 기초적인 문법의 형태는 잘 갖추었으나 미묘하게 목적어가 이상하게 번역되는(?) 그런 결과가 발생하는 것 같았습니다.  
하이퍼 파라미터와 학습 샘플의 수를 잘 조절하여 학습시킨다면 더 좋은 결과를 낼 수 있을것 같습니다!